In [1]:
import pandas as pd

In [2]:
lexicon = pd.read_csv('portilexicon-ud.tsv',sep='\t', header=None)
lexicon.head()

,0,1,2,3
0,a,a,ADP,_
1,a,a,NOUN,Gender=Masc|Number=Sing
2,a,o,DET,Definite=Def|Gender=Fem|Number=Sing|PronType=Art
3,a,o,PRON,Case=Acc|Gender=Fem|Number=Sing|Person=3|PronT...
4,a,o,PRON,Gender=Fem|Number=Sing|Person=3|PronType=Dem


In [3]:
cols = []
for index, row in lexicon[3].str.split('|', expand=True).iterrows():
    cols.extend(list(filter(None, row.str.split('=', expand=True)[0].unique())))
    cols = list(set(cols))
cols.remove("_")
print(cols)
cols_name = [i.lstrip() for i in cols]
print(cols_name)

['NumType', 'Poss', ' Number', 'Number', 'Definite', 'VerbForm', 'Case', 'Abbr', 'Foreign', 'PronType', 'Gender', 'Tense', 'Mood', 'Person']
['NumType', 'Poss', 'Number', 'Number', 'Definite', 'VerbForm', 'Case', 'Abbr', 'Foreign', 'PronType', 'Gender', 'Tense', 'Mood', 'Person']


In [4]:
# constrói dataframe
df_lexicon = pd.DataFrame(columns=['words', 'infinitive'])
df_lexicon['words'] = lexicon[0]
df_lexicon['infinitive'] = lexicon[1]
df_lexicon['pos'] = lexicon[2]
for col_name in cols_name:
    df_lexicon[col_name.lower()] = lexicon[3].str.split(col_name + '=', expand=True)[1].str.split('|', expand=True)[0]

df_lexicon.isna().sum()
df_lexicon.head()
# preenche vazios
#df_lexicon.fillna('NA', inplace=True)
#df_lexicon['gender'].fillna('NA', inplace=True)
#df_lexicon['number'].fillna('NA', inplace=True)

,words,infinitive,pos,numtype,poss,number,definite,verbform,case,abbr,foreign,prontype,gender,tense,mood,person
0,a,a,ADP,None,None,None,None,None,None,None,None,None,None,None,None,None
1,a,a,NOUN,None,None,Sing,None,None,None,None,None,None,Masc,None,None,None
2,a,o,DET,None,None,Sing,Def,None,None,None,None,Art,Fem,None,None,None
3,a,o,PRON,None,None,Sing,None,None,Acc,None,None,Prs,Fem,None,None,3
4,a,o,PRON,None,None,Sing,None,None,None,None,None,Dem,Fem,None,None,3


In [5]:
nomlexpt = pd.read_csv("UDer-1.1-pt-NomLexPT.tsv",sep='\t', header=None)
nomlexpt.head()

,0,1,2,3,4,5,6,7,8,9
0,0.0,abafar#VERB,abafar,VERB,NaN,NaN,NaN,NaN,NaN,{}
1,0.1,abafador#NOUN,abafador,NOUN,NaN,NaN,0.0,Type=Derivation,NaN,{}
2,0.2,abafamento#NOUN,abafamento,NOUN,NaN,NaN,0.0,Type=Derivation,NaN,{}
3,1.0,abaixar#VERB,abaixar,VERB,NaN,NaN,NaN,NaN,NaN,{}
4,1.1,abaixamento#NOUN,abaixamento,NOUN,NaN,NaN,1.0,Type=Derivation,NaN,{}


In [6]:
verbs = nomlexpt[nomlexpt[3] == 'VERB'][2].to_list()
verbs_suffix = ['ar', 'er', 'ir']
verbs_radical = []
for verb in verbs:
    for suffix in verbs_suffix:
        if verb.lower().endswith(suffix.lower()):
            verbs_radical.append(verb[0:-len(suffix)])
            break

In [7]:
import re
def pattern_searcher(search_str:str, search_list:str, verbs:list):
    search_obj = re.search(search_list, search_str)
    if search_obj:
        #returns = search_str[search_obj.start(): search_obj.end()]
        #ret = [verb for verb in verbs if returns in verb][0]
        #return_str = returns + '|' + ret 
        return_str = search_str[search_obj.start(): search_obj.end()]
    else:
        return_str = 'NA'
    return return_str

In [23]:
#pattern_radical = '|'.join(verbs_radical)
pattern_radical = '({})'.format('|'.join(verbs_radical))
df_lexicon['match_root'] = df_lexicon['words'].str.extract(pattern_radical, expand=False).fillna('TESTE')
#df_lexicon['match_root'] = df_lexicon['words'].apply(lambda x: pattern_searcher(search_str=x, search_list=pattern_radical))

In [24]:
pattern_verbs = '({})'.format('|'.join(verbs))
df_lexicon['match_verb'] = df_lexicon['match_root'].str.extract(pattern_verbs, expand=False).fillna('TESTE')
#df_lexicon[['match_root', 'match_verb']] = df_lexicon['match_root'].str.split('|', 1, expand=True)

In [ ]:
df_lexicon

In [ ]:
verbs_match = []
for root in df_lexicon['match_root']:
    for verb in verbs:
        if root in verb:
            verbs_match.append(verb)
        else:
            verbs_match.append('')

In [ ]:
df_lexicon[df_lexicon.match_root.str.startswith('abandon')]

In [ ]:
df_filtered = df_lexicon[(df_lexicon.pos != 'VERB') & (df_lexicon.match_root != '') & (df_lexicon.number != 'Plur') & (df_lexicon.verbform != 'Part')]

In [ ]:
df_filtered

In [ ]:
nomlexpt.head()

In [ ]:
nomlexpt[nomlexpt[3] == 'VERB']

In [ ]:
rels = set()
for index, row in nomlexpt[nomlexpt[3] == 'VERB'].iterrows():
    for index, row2 in df_filtered[df_filtered.match_verb == row[2]].iterrows():
        parent = row[1]
        child = row2['words']
        pos = row2['pos']
        
        if (parent, child, pos) not in rels:
            rels.add((parent, child, pos))
            print(parent, child, pos, sep='\t')
        

In [ ]:
nomlexpt.head(20)

In [ ]:
df_filtered[df_filtered.match_verb == 'abafar']

In [ ]:
nomlexpt[nomlexpt[6] == 0.0]

In [ ]:
nomlexpt[nomlexpt[0] == 1931.0][0].values + 0.1*len(nomlexpt[nomlexpt[6] == 1931.0][0])

In [ ]:
nomlexpt[nomlexpt[6] == 1931.0]

In [10]:
df_lexicon[df_lexicon.match_verb == 'abandonar']

,words,infinitive,pos,numtype,poss,number,definite,verbform,case,abbr,foreign,prontype,gender,tense,mood,person,match_root,match_verb


In [15]:
df_lexicon['match_root'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1226024 entries, 0 to 1226023
Data columns (total 18 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   words       1226024 non-null  object
 1   infinitive  1226024 non-null  object
 2   pos         1226024 non-null  object
 3   numtype     179 non-null      object
 4   poss        40 non-null       object
 5   number      1192172 non-null  object
 6   definite    8 non-null        object
 7   verbform    1078377 non-null  object
 8   case        44 non-null       object
 9   abbr        2384 non-null     object
 10  foreign     135 non-null      object
 11  prontype    295 non-null      object
 12  gender      237613 non-null   object
 13  tense       689892 non-null   object
 14  mood        848287 non-null   object
 15  person      934611 non-null   object
 16  match_root  1226024 non-null  object
 17  match_verb  1226024 non-null  object
dtypes: object(18)
memory usage: 168.4+ MB


In [26]:
df_lexicon['match_verb'].value_counts()

TESTE    1222912
ir          1615
ver          874
impor        196
ler           95
deter         87
expor         87
feder         81
andar         77
Name: match_verb, dtype: int64

In [25]:
from collections import Counter
duplicates = [item for item, count in df_lexicon['match_verb'].value_counts().iteritems()]

dict(Counter(duplicates))

{'TESTE': 1,
 'ir': 1,
 'ver': 1,
 'impor': 1,
 'ler': 1,
 'deter': 1,
 'expor': 1,
 'feder': 1,
 'andar': 1}

In [20]:
pattern_verbs

'(abafar|abaixar|abalar|abalroar|abandonar|abastecer|abater|abdicar|abduzir|abjurar|abnegar|abolir|abominar|abonar|abordar|aborrecer|abortar|abotoar|abranger|abraçar|abreviar|abrigar|abrir|absolutizar|absolver|absorver|abster|abstrair|abundar|abusar|acabar|acalentar|acalmar|acampar|acarear|acarretar|acasalar|acatar|aceder|aceitar|acelerar|acenar|acender|acentuar|acertar|acessar|achar|acidentar|acidificar|acionar|aclamar|aclimatar|acolher|acomodar|acompanhar|aconselhar|acontecer|acoplar|acordar|acossar|acostar|acrescentar|acrescer|activar|actualizar|actuar|aculturar|acumular|acusar|adaptar|adequar|aderir|adestrar|adiar|adicionar|adjudicar|administrar|admirar|admitir|admoestar|adorar|adornar|adotar|adquirir|adubar|adular|adulterar|advertir|advir|advogar|aerar|afastar|aferir|afetar|afiliar|afinar|afirmar|afligir|aflorar|afluir|afogar|aforrar|africanizar|afrontar|agachar|agasalhar|agenciar|agilizar|agir|agitar|aglomerar|aglutinar|agoniar|agradecer|agravar|agredir|agregar|agremiar|agrupar|a

In [22]:
df_lexicon['match_verb'].str.extract(pattern_verbs, expand=False)

0          NaN
1          NaN
2          NaN
3          NaN
4          NaN
          ... 
1226019    NaN
1226020    NaN
1226021    NaN
1226022    NaN
1226023    NaN
Name: match_verb, Length: 1226024, dtype: object